In [1]:
using Revise
using DataFrames
using XLSX
photochemistry_source_dir = "$(@__DIR__)/Photochemistry/src/"
push!(LOAD_PATH, photochemistry_source_dir)
using Photochemistry  # custom module

In [11]:
kJ_to_eV(kj) = kj * 6.242e21
ev_per_molecule(eV_per_mole) = eV_per_mole/6.022e23

#Enthalpy of formation 
const dfH = Dict(:Ar=>0, :CO=>-113.8, :CO2=>-393.1, :D=>221.72, :H=>216.0, :HD=>0.32, :H2=>0, :DCO=>40.945, :HCO=>44.8, :HDO=>-245.28, :H2O=>-238.9, :H2O2=>-130.0,
                :DO2=>6.487, :HO2=>13.4, :DOCO=>185.8, :HOCO=>183.97, :N2=>0, :O=>246.8, :O1D=>246.8, :O2=>0, :O3=>141.80, :OD=>37.23, :OH=>38.4, :HDO2=>-140.242, 
                :C=>711.2, :N=>470.8, :NO=>89.8, :Nup2D=>470.8, :CO2pl=>935.7, :DCO2pl=>594.9, :HCO2pl=>589.0, :Opl=>1560.7, :O2pl=>1164.7, :Arpl=>166.40, 
                :ArDpl=>1176.9, :ArHpl=>1165.2, :Cpl=>1797.6, :CHpl=>1619.1, :COpl=>1238.3, :Dpl=>1540.320, :Hpl=>1528.0, :HDpl=>1496.793, :H2pl=>1488.3, 
                :H2Dpl=>1118.1, :H3pl=>1107.0, :HDOpl=>987.7, :H2Opl=>977.9, :H3Opl=>597.0, :H2DOpl=>1119.6, :HO2pl=>1108.5, :DCOpl=>833.9,
                :HCOpl=>825.6, :DOCpl=>972.6, :HOCpl=>963.0, :HNOpl=>1074.4, :Npl=>1873.1, :NHpl=>1359, :N2pl=>1503.3, :N2Dpl=>1045.9, :N2Hpl=>1035.5,
                :NOpl=>984.0, :ODpl=>1305.6, :OHpl=>1292.7, :E=>0);

const escape_eV = Dict(:H=>escape_energy(1), :D=>escape_energy(2), :H2=>escape_energy(2), :HD=>escape_energy(3))
    

Dict{Symbol, Float64} with 4 entries:
  :H  => 0.130302
  :D  => 0.260603
  :H2 => 0.260603
  :HD => 0.390905

In [36]:
function enthalpy_of_reaction(reactants, products)
    
    dfH_products = 0
    dfH_reactants = 0

    for p in products
        dfH_products += dfH[p]
    end
    for r in reactants
        dfH_reactants += dfH[r]
    end

    enthalpy_of_rxn = dfH_products - dfH_reactants
    
    m = 0 # this block adds up the total mass that needs to escape, allows for reactions that produce, e.g., both H and D
    flag = 0
    if :H in products
        m += 1
        flag += 1
    end
    if :H2 in products
        m += 2
        flag += 1
    end
    if :D in products
        m += 2
        flag += 1
    end
    if :HD in products
        m += 3
        flag += 1
    end
        
    enthalpy_in_ev = ev_per_molecule(kJ_to_eV(-1*(enthalpy_of_rxn)))
    excess_energy = round(enthalpy_in_ev - escape_energy(m), digits=2)

    endo_exo = excess_energy > 0 ? "exothermic" : "endothermic"
    
    # println("Reaction $(format_chemistry_string(reactants, products))")
    # if flag > 1
    #     println("Flag! Reaction produces two hot molecules")
    # end
    # println("Raw enthalpy is $(enthalpy_in_ev) eV")
    # println("Mass to escape $(m), so need $(m*0.13) excess eV")
    # println("excess energy $(excess_energy)")
    # println("It is: $(endo_exo)")
    # println()
    
    return endo_exo, excess_energy
end

function get_product_and_reactant_cols(df)
    # Determine how many reactant and product columns there are 
    possible_Rcols = ["R1", "R2", "R3"]
    possible_Pcols = ["P1", "P2", "P3"]
    rcols = Symbol.(possible_Rcols[possible_Rcols .∈ Ref(names(df))])
    pcols = Symbol.(possible_Pcols[possible_Pcols .∈ Ref(names(df))])
    
    return rcols, pcols
end

function enthalpies_of_reactions(df; species=[:H, :D], insert_i=nothing)
    
    if insert_i != nothing
        insertcols!(df, insert_i, :hotH2=>["" for i in collect(1:size(df)[1])])
        insertcols!(df, insert_i+1, :hotHD=>["" for i in collect(1:size(df)[1])])
    end
    
    rcols, pcols = get_product_and_reactant_cols(df)
    
    endo_count = 0
    exo_count = 0
    
    for row in eachrow(df)
        reactants = [Symbol(row.:($r)) for r in rcols]
        products = filter!(x->x!=:none, [Symbol(row.:($p)) for p in pcols])
        if any(x->!(x in keys(dfH)), union(reactants, products))
            continue
        end

        if any(x->x in products, species)
            exo_or_endo, excess_energy = enthalpy_of_reaction(reactants, products)
            row.excessE = excess_energy
            
            if exo_or_endo == "exothermic"
                exo_count += 1
                row.NTEscape = "Yes"

                if :D in products
                    row.hotD = "Yes"
                end
                if :HD in products
                    row.hotHD = "Yes"
                end
                
                if :H in products
                    row.hotH = "Yes"
                end
                if :H2 in products
                    row.hotH2 = "Yes"
                end
            else 
                # println("$(format_chemistry_string(reactants, products)) is endothermic")
                # println()
                endo_count += 1
                row.NTEscape = "No"
                if :D in products
                    row.hotD = ""
                end
                if :HD in products
                    row.hotHD = ""
                end
                
                if :H in products
                    row.hotH = ""
                end
                if :H2 in products
                    row.hotH2 = ""
                end
            end
        end
    end
    
    println("Exothermic: $(exo_count), Endothermic: $(endo_count)")
    return df
end

function modify_rxn_spreadsheet(spreadsheet; spc=[:H, :D], insert_i=nothing)
    xf = XLSX.readxlsx(spreadsheet)
    original_sheets = XLSX.sheetnames(xf)
    
    new_file = "MOLEC_NT_$(spreadsheet)"#"REACTION_NETWORK.xlsx"
    
    for (j, sheet) in enumerate(original_sheets)
        df = DataFrame(XLSX.readtable(spreadsheet, sheet)...);
               
        # Now process:        
        if sheet in ["Ion reactions", "Photodissociation", "Photoionization"]
            # Replace any missing values so we don't get rid of all our reactions before processing:
            rcols, pcols = get_product_and_reactant_cols(df)
            for r in rcols
                replace!(df.:($r), missing=>"none")
            end
            for p in pcols
                replace!(df.:($p), missing=>"none")
            end
            
            df_to_write = enthalpies_of_reactions(df; species=spc, insert_i=insert_i[j])
        else
            df_to_write = df
        end
        println()
        log_reactions(df_to_write, sheet, new_file)
    end
end

escape_energy(massmult) = kJ_to_eV((0.5 * massmult * (1.67e-27 #=kg=#) * (5000 #=m/s=#)^2) / 1000 #=kJ/J=#)

escape_energy (generic function with 1 method)

In [6]:
# modify_rxn_spreadsheet("REACTION_NETWORK_NOENTHALPY.xlsx"; insert_i=[0,7,8,8,0,0])

In [37]:
modify_rxn_spreadsheet("REACTION_NETWORK_BEFOREH2.xlsx"; spc=[:H2, :HD, :H, :D], insert_i=[0,11,12,12,0,0,0])


Exothermic: 146, Endothermic: 22

Exothermic: 0, Endothermic: 16

Exothermic: 0, Endothermic: 8



